# OpenGeoHub Summer School 2022 - Introduction to GeoPandas and its Python ecosystem

**Martin Fleischmann**

<sup>1</sup>Geographic Data Science Lab, University of Liverpool, UK<br>
<sup>2</sup>Urban and Regional Laboratory, Charles University, CZ<br>
<sup>3</sup>UrbanDataLab AG, Zürich, CH

31/08/2022, Siegburg

## Setup

Follow the Readme to set-up the environment correctly. You should have these packages installed:

```
- geopandas
- dask-geopandas
- pyogrio
- pyarrow
- python-graphviz
- esda
- momepy
- libpysal
- dask-labextension # optionally, if using JupyterLab
```

## What is GeoPandas

GeoPandas, as the name suggests, extends the popular data science library [pandas](https://pandas.pydata.org) by adding support for geospatial data.

The core data structure in GeoPandas is the `geopandas.GeoDataFrame`, a subclass of `pandas.DataFrame`, that can store geometry columns and perform spatial operations. The `geopandas.GeoSeries`, a subclass of `pandas.Series`, handles the geometries. Therefore, your `GeoDataFrame` is a combination of `pandas.Series`, with traditional data (numerical, boolean, text etc.), and `geopandas.GeoSeries`, with geometries (points, polygons etc.). You can have as many columns with geometries as you wish; there's no limit typical for desktop GIS software.

![geodataframe schema](fig/dataframe.svg)

Each `GeoSeries` can contain any geometry type (you can even mix them within a single array) and has a `GeoSeries.crs` attribute, which stores information about the projection (CRS stands for Coordinate Reference System). Therefore, each `GeoSeries` in a `GeoDataFrame` can be in a different projection, allowing you to have, for example, multiple versions (different projections) of the same geometry.

Only one `GeoSeries` in a `GeoDataFrame` is considered the _active_ geometry, which means that all geometric operations applied to a `GeoDataFrame` operate on this _active_ column.

But when we talk about GeoPandas, we first have to talk about pandas.

## What is pandas?

Once again, the best place to ask this question is the documentation.

> `pandas` is an open source library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

What that means in practice? Pandas provides two key data structures - a `Series` and a `DataFrame`. Since you will mostly work with DataFrames, composed of Series, we can start with that.

![dataframe](fig/01_table_dataframe.svg)

### Reading and writing

We can start quickly illustrating pandas in practice by reading some files. While pandas can read many file formats, we can keep it simple and read a CSV.

![io](fig/02_io_readwrite.svg)

In [ ]:
import pandas

We will work with some demographic characteristics in Liverpool, borrowed from a [Geographic Data Science](https://darribas.org/gds_course/content/bB/lab_B.html) course by [Dani Arribas-Bel](http://twitter.com/darribas).

In [ ]:
liv_pop = pandas.read_csv(
    "https://darribas.org/gds_course/content/data/liv_pop.csv",  # path to the file
    index_col="GeographyCode",  # column to use as an index
)

We can check how the data looks like by simply typing its name.

In [ ]:
liv_pop

Each column or row is a `Series` object, which we can access based on a name, a location or other means.

![subset](fig/03_subset_columns_rows.svg)

In [ ]:
liv_pop["Middle East and Asia"]

We can further use the columns to do algebra on them or turn them into boolean masks.

In [ ]:
liv_pop["Middle East and Asia"] > 500

Where boolen masks can be used to filter the DataFrame.

In [ ]:
liv_pop[liv_pop["Middle East and Asia"] > 500]

And combine masks along rows with explicit list of columns.

In [ ]:
liv_pop.loc[
    liv_pop["Middle East and Asia"] > 500, ["Africa", "The Americas and the Caribbean"]
]

Of course, we can also combine columns to create a new one. What is the total population of each LSOA?

![new column](fig/05_newcolumn_2.svg)

In [ ]:
total = liv_pop.sum(axis=1)
total

In [ ]:
liv_pop["Total"] = total
liv_pop.head()

We can also quickly visualize the data. (But not on a map yet, that is where GeoPandas comes in.)

![plot](fig/04_plot_overview.svg)

In [ ]:
liv_pop["Total"].plot.hist()

In [ ]:
liv_pop.plot.scatter("Africa", "Middle East and Asia")

**Let's go back to GeoPandas!**

If you look again at the diagram shown above (and here once again), you now undestand that both `index` and `data` are a practically a `pandas.DataFrame`. But what else is happenning there?

![geodataframe schema](fig/dataframe.svg)


## The pieces we bring together

Any software that has an ambition to properly deal with geospatial data needs to cover four aspects:

- *data*
    - We just learned that this part is covered by `pandas`.
- *geometry*
    - Handling of the actual geometries and operations on them. In FOSS, there is one leading engine, `GEOS`, but given it is written in C++, we need a Python wrapper. That is called `shapely`.
- *projections*
    - We need to know which place on the Earth (or other planet) each coordinates refer to. That is what projections, or Coordinate Reference Systems (CRS), take care of. The main FOSS engine here is `PROJ`, again written in C++ and in Python wrapped by `pyproj`.
- *reading and writing files*
    - Geospatial world knows a lot of file formats to store the data, from Shapefile and GeoJSON to GeoParquet. The best way to read and write into them is to use `GDAL/OGR`, another C++ library, which is avaliable either via `fiona` or recently via `pyogrio`.

![ecosystem diagram](fig/ecosystem.png)

Let's see all of them in action within GeoPandas.

## GeoPandas in action

First, we need to read some data.

### Reading files

Assuming you have a file containing both data and geometry (e.g. GeoPackage, GeoJSON, Shapefile), you can read it using `geopandas.read_file()`, which automatically detects the filetype and creates a `GeoDataFrame`. This can use either `fiona` or `pyogrio` as an engine (as of GeoPandas 0.11) interfacing GDAL/OGR to do the heavy lifting. We use `pyogrio` as a newer, faster option.

We will again borrow data from the [GDS course](https://darribas.org/gds_course/content/bC/lab_C.html#cities) we used before. This time we load the boundaries of Spanish cities.

In [ ]:
import geopandas

cities = geopandas.read_file("https://ndownloader.figshare.com/files/20232174")
cities

### Writing files

To write a `GeoDataFrame` back to file use `GeoDataFrame.to_file()`. The file is format automatically inferred from the suffix, but you can specify your own with the `driver` keyword. When no suffix is given, GeoPandas expects you want a folder with ESRI Shapefile.

In [ ]:
cities.to_file("cities.gpkg")

### Geometries

Geometries within the *geometry* column are `shapely` objects. GeoPandas itself is not creating the object but leverages the existing ecosystem (note that there is a significant overlap of the team writing both to ensure synergy).

In [ ]:
cities.geometry

In [ ]:
type(cities.geometry.loc[0])

In [ ]:
cities.geometry.loc[0]

In [ ]:
cities.geometry.loc[0].wkt

But without an assigned CRS, we don't know where on the Earth this shape lies. Therefore, each geometry column has (optionally) assigned one.

In [ ]:
cities.crs

In [ ]:
type(cities.crs)

You can see how the whole ecosystem comes together.

### Simple accessors and methods

Now we have our `GeoDataFrame` and can start working with its geometry. 

Since there was only one geometry column in the Spanish cities dataset, this column automatically becomes the _active_ geometry and spatial methods used on the `GeoDataFrame` will be applied to the `"geometry"` column.

#### Measuring area

To measure the area of each polygon, access the `GeoDataFrame.area` attribute, which returns a `pandas.Series`. Note that `GeoDataFrame.area` is just `GeoSeries.area` applied to the _active_ geometry column.

In [ ]:
cities["area"] = cities.area
cities["area"]

#### Getting polygon boundary and centroid

To get the boundary of each polygon (LineString), access the `GeoDataFrame.boundary`:

In [ ]:
cities["boundary"] = cities.boundary
cities["boundary"]

Since we have saved boundary as a new column, we now have two geometry columns in the same `GeoDataFrame`.

We can also create new geometries, which could be, for example, a buffered version of the original one (i.e., `GeoDataFrame.buffer(10)`) or its centroid:

In [ ]:
cities["centroid"] = cities.centroid
cities["centroid"]

#### Measuring distance

We can also measure how far each centroid is from the first centroid location.

In [ ]:
first_point = cities["centroid"].iloc[0]
cities["distance"] = cities["centroid"].distance(first_point)
cities["distance"]

Note that `geopandas.GeoDataFrame` is a subclass of `pandas.DataFrame`, so we have all the pandas functionality available to use on the geospatial dataset — we can even perform data manipulations with the attributes and geometry information together.

For example, to calculate the average of the distances measured above, access the `"distance"` column and call the `mean()` method on it:

In [ ]:
cities["distance"].mean()

### Making maps

GeoPandas can also plot maps, so we can check how the geometries appear in space. To plot the active geometry, call `GeoDataFrame.plot()`. To color code by another column, pass in that column as the first argument. In the example below, we plot the active geometry column and color code by the `"area"` column. We also want to show a legend (`legend=True`).

In [ ]:
cities.plot("area", legend=True, figsize=(12, 8))

<div class="alert alert-info">
Other resources for plotting

Want to know more on static plots? Check [this chapter](https://darribas.org/gds_course/content/bC/lab_C.html#styling-plots) of the GDS course or [the GeoPandas documentation](https://geopandas.org/en/stable/docs/user_guide/mapping.html).
</div>

You can also explore your data interactively using `GeoDataFrame.explore()`, which behaves in the same way `plot()` does but returns an interactive (leaflet.js) map instead.

In [ ]:
cities.explore("area", legend=False)

Switching the active geometry (`GeoDataFrame.set_geometry`) to centroids, we can plot the same data using point geometry.

In [ ]:
cities = cities.set_geometry("centroid")
cities.explore()

And we can also layer both `GeoSeries` on top of each other. We just need to use one plot as an axis for the other.

In [ ]:
m = cities["geometry"].explore()
cities["centroid"].explore(m=m, color="black")

Now we set the active geometry back to the original `GeoSeries`.

In [ ]:
cities = cities.set_geometry("geometry")

### Spatial predicates

Spatial predicates tell us what is the spatial relation between two geometries. Are they equal, do they intersects, overlap, is one wihtin another?

For this part we use a toy datasets that come with GeoPandas, so we just need to retrive their location within the package installation.

In [ ]:
world_countries = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world_cities = geopandas.read_file(geopandas.datasets.get_path('naturalearth_cities'))

In [ ]:
m = world_countries.explore()
world_cities.explore(m=m, color='red')

Let's first create some small toy spatial objects:

A polygon <small>(note: we use `.item()` here to to extract the scalar geometry object from the GeoSeries of length 1)</small>:

In [ ]:
belgium = world_countries.loc[world_countries['name'] == 'Belgium', 'geometry'].item()

Two points:

In [ ]:
paris = world_cities.loc[world_cities['name'] == 'Paris', 'geometry'].item()
brussels = world_cities.loc[world_cities['name'] == 'Brussels', 'geometry'].item()

And a linestring:

In [ ]:
from shapely.geometry import LineString

line = LineString([paris, brussels])

Let's visualize those 4 geometry objects together:

In [ ]:
geopandas.GeoSeries([belgium, paris, brussels, line], crs=world_cities.crs).explore()

You can recognize the abstract shape of Belgium.

Brussels, the capital of Belgium, is thus located within Belgium. This is a spatial relationship, and we can test this using the individual shapely geometry objects as follow:

In [ ]:
brussels.within(belgium)

And using the reverse, Belgium contains Brussels:

In [ ]:
belgium.contains(brussels)

On the other hand, Paris is not located in Belgium:

In [ ]:
belgium.contains(paris)

In [ ]:
paris.within(belgium)

The straight line we draw from Paris to Brussels is not fully located within Belgium, but it does intersect with it:

In [ ]:
belgium.contains(line)

In [ ]:
line.intersects(belgium)

#### Spatial relationships with GeoDataFrames

The same methods that are available on individual `shapely` geometries as we have seen above, are also available as methods on `GeoSeries` / `GeoDataFrame` objects.

For example, if we call the `contains` method on the world dataset with the `paris` point, it will do this spatial check for each country in the `world` dataframe:

In [ ]:
world_countries.contains(paris)

Because the above gives us a boolean result, we can use that to filter the dataframe:

In [ ]:
world_countries[world_countries.contains(paris)]

We could also do the same based on a query over the spatial index. Custom queries on spatial index using `query` and esp. `query_bulk` are often much faster but are also considered an advanced usage. Since GeoPandas wraps them in spatial joins covering most of the cases, you may not even need to access `sindex` directly yourself.

In [ ]:
world_countries.iloc[world_countries.sindex.query(paris, "within")]

### Spatial join

One of he most typical geospatial tasks is a spatial join. Let's go back to our dataset with the Spanish city boundaries and try to figure out which of these cities fall into Catalonia region and which province they belong to.

First, we need a data representing Catalonia. We get one directly from the Catalonian open data portal.

In [ ]:
catalonia = geopandas.read_file(
    "https://analisi.transparenciacatalunya.cat/api/geospatial/ghr8-wp3h?method=export&format=Shapefile",
)
catalonia

We can check the CRS, because for spatial join, we need to be sure that both GeoDataFrames are using the same (but don't worry, GeoPandas would warn you in case of a CRS mismatch).

In [ ]:
catalonia.crs

In [ ]:
cities.crs

Since this one is different, we can re-project the geometries to the CRS of `cities`.

In [ ]:
catalonia = catalonia.to_crs(cities.crs)

Now we can do the spaital join using the `sjoin` method. That by default uses the `intersects` predicate but you can use any of them.

In [ ]:
cities_within_catalonia = cities.sjoin(catalonia)

Let's see the result.

In [ ]:
len(cities_within_catalonia)

In [ ]:
cities_within_catalonia.head()

In [ ]:
cities_within_catalonia.explore(
    "nom_prov",
    tiles="Stamen Toner",
)

Since GeoDataFrames are still based on pandas DataFrames, we can readily use pandas functionality, like `groupby` on the result.

In [ ]:
cities_within_catalonia.groupby("nom_prov").area.sum()

### Overlay

In some cases, we may want to create new geometries based on their spatial relationships between existing geometries. We call these _overlay_ operations.

Let's assume that we are interested in areas that are 10km around a centroid of each city.

In [ ]:
buffered_centroids = cities['centroid'].buffer(10_000)

In [ ]:
buffered_centroids.explore()

So we can ask for an intersection between the buffer and city boundaries.

![intersection](https://geopandas.org/en/stable/_images/binary_geo-intersection.svg)

In [ ]:
cities.overlay(buffered_centroids.to_frame(), how="intersection").explore()

Or we may want to do the union of the two to get area covered either by distance or the boundary.

![union](https://geopandas.org/en/stable/_images/binary_geo-union.svg)

In [ ]:
cities.overlay(buffered_centroids.to_frame(), how="union").explore()

Or do the difference. Find all parts of boundaries that are further away than 10km from each centroid.

![difference](https://geopandas.org/en/stable/_images/binary_geo-difference.svg)

In [ ]:
cities.overlay(buffered_centroids.to_frame(), how="difference").explore()

And finally, we may as which parts areas are either wihtin 10km or wihtin a boundary but not both.

![difference](https://geopandas.org/en/stable/_images/binary_geo-symm_diff.svg)

In [ ]:
cities.overlay(buffered_centroids.to_frame(), how="symmetric_difference").explore()

## The ecosystem

We can now move beyond GeoPandas, because while being useful itself, its main benefit is that it enables others to build on top of a robust basis. We can take an example of PySAL, Python Spatial Analysis Library. PySAL is a federation of 21 individual packages covering everything from basic spatial weights matrices to spatial regression, optimisation, point pattern analysis or dasymetric mapping.

Today, we will look at two of them to learn how to work with spatial weights and how to measure local index of spatial autocorrelatoin (LISA).

Let's go back to our LSOA-based data from Liverpool. This time, we load the geometries using geopandas.

In [ ]:
liv_lsoa = geopandas.read_file(
    "https://darribas.org/gds_course/content/data/liv_lsoas.gpkg",
)
liv_lsoa

In [ ]:
liv_lsoa.explore()

### Spatial weights

Spatial weights can be created in many ways based on many conditions. We are now intersted in the most common one - contiguity. We simply want to encode, which geometries are neighbours of which.

To generate such a matrix, we can use the `weights` module of `libpysal` and its `Queen` contiguity implementation.

In [ ]:
from libpysal import weights

w_queen = weights.Queen.from_dataframe(liv_lsoa, idVariable="LSOA11CD")
w_queen

Above, we have used the LSOA code to encode the spatial weights. To make it a bit easier below, let's set the same variable as an index.

In [ ]:
liv_lsoa = liv_lsoa.set_index("LSOA11CD")
liv_lsoa

Now we can explore what `w_queen` object actually contains. We can pick one LSOA, say `E01006690` and check its neighbors.

In [ ]:
w_queen["E01006690"]

The number `1.0` encodes the strength, because weights can be more complex than simple boolean "is neighbor" or "is not". It can potentially be weighted based on the lenght of the common boundary.

We can also quickly check how many neighbours it has.

In [ ]:
w_queen.cardinalities["E01006524"]

And if it is more or less than the average.

In [ ]:
w_queen.mean_neighbors

Let's check this on the map.

In [ ]:
lsoa = "E01006690"

m = liv_lsoa.loc[[lsoa]].explore(color="red")
liv_lsoa.loc[w_queen[lsoa].keys()].explore(m=m)

We can also use other ways of creating weights. For example K-nearest neighbors or a distance band.

We can find 10 nearest neighbors (based on the centroid).

In [ ]:
w_knn = weights.KNN.from_dataframe(liv_lsoa, k=10, ids=liv_lsoa.index)

In [ ]:
lsoa = "E01006690"

m = liv_lsoa.loc[[lsoa]].explore(color="red")
liv_lsoa.loc[w_knn[lsoa].keys()].explore(m=m)

Or all neighbours within 2 kilometres.

In [ ]:
w_2km = weights.DistanceBand.from_dataframe(liv_lsoa, threshold=2000, ids=liv_lsoa.index)

In [ ]:
lsoa = "E01006690"

m = liv_lsoa.loc[[lsoa]].explore(color="red")
liv_lsoa.loc[w_2km[lsoa].keys()].explore(m=m)

### Spatial lag

Now, remember that we also had some data for these areas. Let's compare the two dataframes.

In [ ]:
liv_lsoa

In [ ]:
liv_pop

One has geometries but no data, the other has data but no geometries. But both share the LSOA code we can use to merge them.

In [ ]:
liv_lsoa = liv_lsoa.merge(liv_pop, left_on="LSOA11CD", right_on="GeographyCode")

In [ ]:
liv_lsoa

One of the very useful things we can do with spatial weights is to calculate a spatial lag. That is, in principle, a mean of values from neighbouring geometries.

We can measure it easily using `lag_spatial` function. But before that, we need to transform the weights so the all weights (the numbers) for each geometry sum up to one. Otherwise we would measure the sum of instead of mean.

Weights before the transformation:

In [ ]:
w_queen.weights['E01006512']

Row-wise transformation:

In [ ]:
w_queen.transform = "R"

Weights after the transformation:

In [ ]:
w_queen.weights['E01006512']

Now we can measure the lag. For example of the `"Middle East and Asia"` column.

In [ ]:
w_queen_score = weights.lag_spatial(w_queen, liv_lsoa["Middle East and Asia"])

Let's see the result.

In [ ]:
w_queen_score[:10]

We can assign it to the dataframe as a column.

In [ ]:
liv_lsoa["Middle East and Asia lagged"] = w_queen_score

### Moran plot

With the spatial lag, we can start asking some spatial questions. Like, is there a spatial autocorrelation? The simplest way of getting a sense on that is to plot the original variable against its spatial lag, so called Moran plot.

In [ ]:
liv_lsoa.plot.scatter(
    "Middle East and Asia", "Middle East and Asia lagged", figsize=(8, 8)
)

When there is a relationship between the two, we can assume spatial autocorrelation of the variable. And here it seems that there is a strong one.

### LISA

We can explore that further, using the Local Indicator of Spatial Autocorrelation (LISA), implemented in another of PySAL's packages called `esda`.

In [ ]:
from esda.moran import Moran_Local

Before we do the actual measuring, we should normalise the values as some of the LSOAs have larger total population than the others and it would skew the output. We can rather look at proportions. 

In [ ]:
liv_lsoa["Middle East and Asia proportion"] = (
    liv_lsoa["Middle East and Asia"] / liv_lsoa["Total"]
)

We use our existing spatial weights, pass the variable and measure the LISA based on Moran's I using the `Moran_Local` class.

In [ ]:
moran_loc = Moran_Local(liv_lsoa["Middle East and Asia proportion"], w_queen)

We can now explore the object. 

First, a LISA quadrant encoding if the geometry has high values and is surrounded by other of high values or any other combination of high and low values.

In [ ]:
moran_loc.q

We have a simulated P value indicating significance of the result for each geometry.

In [ ]:
moran_loc.p_sim

In [ ]:
moran_loc

That is enough for us to plot the result and see whether there are any hotspots where Asian and Middle East population tend to live or any cold spots they are not present in.

For easier reading we can relabel the quadrants with self-explanatory labels.

In [ ]:
moran_labels = {
    1: "High-high",
    2: "Low-high",
    3: "Low-low",
    4: "High-low",
}

And now we can just assign the quadrants and P-values as columns, filter based on significance and look at the map.

In [ ]:
liv_lsoa["moran_q"] = moran_loc.q
liv_lsoa["moran_q"] = liv_lsoa["moran_q"].map(moran_labels)
liv_lsoa["moran_p"] = moran_loc.p_sim
liv_lsoa.loc[liv_lsoa["moran_p"] > 0.05, "moran_q"] = None  # filter by significance

In [ ]:
liv_lsoa.explore("moran_q")

We can quilckly compare the quadrants to the actual distribution, to see if it "makes sense".

In [ ]:
liv_lsoa.explore("Middle East and Asia proportion")

We may want to take these clusters and get only their boundaries. In practice, we want to _union_ all the geometries belonging to a single class. That is where GeoPandas is comes useful again.

In [ ]:
liv_lsoa.head()

GeoPandas has a built in method for this spatial groupby, called `dissolve`.

In [ ]:
clusters = liv_lsoa.dissolve("moran_q")
clusters

In [ ]:
clusters.explore()

Right now, we have mutli polygons, one geometry per class. Let's say we want each location on its own. We want to _explode_ the geometries:

In [ ]:
clusters_individual = clusters.explode(index_parts=True)
clusters_individual

### A small exercise

You can later check if we can see similar patterns for other demographic groups, how do they change based on different spatial weights types (Queen, KNN, DistanceBand...). You can also play with overlay on top of resuling dissolved polygons to find the most common areas that belong to different quadrants (or you can try to do that based on attributes and compare).

## Going big

We are entering the last part of the tutorial. This even may be a homework for you, depends on the time :).

Everything we ave used so far was running in a single thread on your computers. Given you probably have 4 or more cores on your CPU, only one was running, other were idle.

For these situations, and for those when your data don't fit your RAM, the Python ecosystem offers a solution called Dask and the combination of Dask and GeoPandas called Dask-GeoPandas. 

### What is Dask?

From the docs:

> Dask provides advanced parallelism and distributed out-of-core computation with a dask.dataframe module designed to scale pandas. Since GeoPandas is an extension to the pandas DataFrame, the same way Dask scales pandas can also be applied to GeoPandas.

For more, see the [Dask tutorial](https://tutorial.dask.org).

### Dask-GeoPandas

We use the power of Dask and GeoPandas to run geospatial processing in parallel right now, but you can even use it to run the code on a distributed cluster and even out-of-core, when your data don't fit the memory.


In [ ]:
import dask_geopandas

That in practice means that we split the GeoDataFrame into multiple partitions and each is processed by a single thread. And some communication between the workers that is controlled by Dask.

In [ ]:
liv_ddf = dask_geopandas.from_geopandas(liv_lsoa, npartitions=4)
liv_ddf

Since it is done lazily as Dask expects the data may be too large, we don't see the values here. For now.

The API works nearly exactly the same as we have seen above. It is just split.

In [ ]:
perimeter = liv_ddf.length
perimeter.visualize()

In [ ]:
liv_ddf["perimeter"] = perimeter

When we call `compute()`, Dask runs all the tasks and returns the result. Same we would get from GeoPandas, just a bit faster.

In [ ]:
result = liv_ddf.compute()
result

How faster? Depends on the machine and operation, of course, but here is a simple example.

The GeoDataFrame used above is a bit small to see any benefit from parallelization using dask (as the overhead of the task scheduler is larger than the actual operation on such a tiny dataframe), so let’s create a bigger point GeoSeries:

In [ ]:
import numpy

N = 10_000_000
points = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(numpy.random.randn(N), numpy.random.randn(N))
)

And creating the dask-geopandas version of this series:

In [ ]:
dpoints = dask_geopandas.from_geopandas(points, npartitions=16)

A single polygon for which we will check if the points are located within this polygon:

In [ ]:
import shapely.geometry

box = shapely.geometry.box(0, 0, 1, 1)

Let’s compare the time it takes to compute this:

In [ ]:
%timeit points.within(box)

In [ ]:
%timeit dpoints.within(box).compute()

## Resources

This tutorial is more closely or more loosely based on the following resources:

- [Introduction to GeoPandas](https://geopandas.org/en/latest/getting_started/introduction.html) by Martin Fleischmann and GeoPandas contributors
- [GeoPandas API reference](https://geopandas.org/en/stable/docs/reference.html) by GeoPandas contributors
- [Getting started (pandas)](https://pandas.pydata.org/docs/getting_started/index.html) by Stijn Van Hoey and pandas contributors
- Arribas-Bel, (2019). A course on Geographic Data Science. Journal of Open Source Education, 2(14), 42, [https://doi.org/10.21105/jose.00042](https://doi.org/10.21105/jose.00042)
- Introduction to geospatial data analysis with GeoPandas and the PyData stack bu Joris van den Bossche (https://github.com/jorisvandenbossche/geopandas-tutorial)
- Fleischmann, M., Feliciotti, A. and Kerr, W. (2022), Evolution of Urban Patterns: Urban Morphology as an Open Reproducible Data Science. Geogr Anal, 54: 536-558. [https://doi.org/10.1111/gean.12302](https://doi.org/10.1111/gean.12302)
- datos.gob.es [Administrative province boundaries in Catalonia](https://datos.gob.es/en/catalogo/a09002970-limites-administativos-provinciales-de-catalunya)